In [1]:
import sqlite3
import datetime
conn = sqlite3.connect('/home/arcyleung/Projects/frigate/config/frigate.db')
cursor = conn.cursor()

print("Tables:")
for row in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(row[0])

# Define the start time (now - 24 hours)
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(hours=24)

# print("schema:", cursor.execute("SELECT * FROM event").description)

# Read the "event" table and filter records with "start_time" within the last 24 hours
print("start_time:", start_time.timestamp())
print("end_time:", end_time.timestamp())
cursor.execute("SELECT * FROM event WHERE start_time >= :start_time AND start_time <= :end_time", {'start_time': start_time.timestamp(), 'end_time': end_time.timestamp()})
events = cursor.fetchall()

# Print the filtered events
print(len(events), "events found")
ids = [ f"{event[2]}-{event[0]}" for event in events]
# for event in events:
#     print(event)
#     print(event[0], event[1])
print(ids)

conn.close()


Tables:
migratehistory
recordings
timeline
event
regions
start_time: 1722739350.902436
end_time: 1722825750.902436
136 events found
['front-1722818479.36576-2y22cv', 'front-1722818476.88936-7k21b5', 'front-1722815507.428464-wxpzlj', 'front-1722815501.166458-xp6bqb', 'front-1722814631.375881-wza9un', 'front-1722814627.028648-0yreg8', 'front-1722812710.037991-93xrxj', 'front-1722812696.283096-oxk84h', 'front-1722812689.273292-jmxlz6', 'front-1722811510.775395-k4dzld', 'front-1722811506.360419-7ubo8b', 'front-1722811330.364074-f4e2rv', 'front-1722810597.211508-od8wmw', 'front-1722810399.889922-2091ah', 'front-1722810399.83827-1pid0u', 'front-1722810397.028705-sf940v', 'front-1722810250.32689-727phi', 'front-1722810217.716625-7ri4uz', 'front-1722810217.201836-ld6ox4', 'front-1722810214.260108-3ac7jr', 'front-1722810212.726919-jufj46', 'front-1722810212.225093-cduplt', 'front-1722810211.889601-2sxwfg', 'front-1722809373.755573-w573qf', 'front-1722809371.221085-g3jfzd', 'front-1722809355.651

In [2]:
import os

def get_filenames(path, ids=None):
    filenames = []
    if ids is not None:
        for id in ids:
            filenames.append(os.path.join(path, id) + "-clean.png")
    else:
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith('-clean.png'):
                    filenames.append(os.path.join(root, file))
    return filenames

filenames = get_filenames('/tmp/storage/clips', ids)
print(filenames)

['/tmp/storage/clips/front-1722818479.36576-2y22cv-clean.png', '/tmp/storage/clips/front-1722818476.88936-7k21b5-clean.png', '/tmp/storage/clips/front-1722815507.428464-wxpzlj-clean.png', '/tmp/storage/clips/front-1722815501.166458-xp6bqb-clean.png', '/tmp/storage/clips/front-1722814631.375881-wza9un-clean.png', '/tmp/storage/clips/front-1722814627.028648-0yreg8-clean.png', '/tmp/storage/clips/front-1722812710.037991-93xrxj-clean.png', '/tmp/storage/clips/front-1722812696.283096-oxk84h-clean.png', '/tmp/storage/clips/front-1722812689.273292-jmxlz6-clean.png', '/tmp/storage/clips/front-1722811510.775395-k4dzld-clean.png', '/tmp/storage/clips/front-1722811506.360419-7ubo8b-clean.png', '/tmp/storage/clips/front-1722811330.364074-f4e2rv-clean.png', '/tmp/storage/clips/front-1722810597.211508-od8wmw-clean.png', '/tmp/storage/clips/front-1722810399.889922-2091ah-clean.png', '/tmp/storage/clips/front-1722810399.83827-1pid0u-clean.png', '/tmp/storage/clips/front-1722810397.028705-sf940v-clean.

In [3]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True, load_in_4bit=True, use_flash_attention_2=False)
# model.to("cuda:0")



/home/arcyleung/Projects/llm-nvr/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.01s/it]


In [4]:


# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image")
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "Describe any people in the scene, what clothing they are wearing, orientation with other objects, anything they are holding and what do they appear to be doing. Also describe prominent vehicles or legible text, and do not mention anything that shoule be ordinary."},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)


In [5]:

from PIL import Image
import datetime
import json

def transcribe_one(filename, responses):
    image = Image.open(filename)
    inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

    # autoregressively complete prompt
    output = model.generate(**inputs, max_new_tokens=150)

    text = processor.decode(output[0], skip_special_tokens=True)
    responses[filename] = text[len(prompt) - 5:]

responses = {}
today = datetime.datetime.now().isoformat()

for idx, filename in enumerate(filenames):
    print(f"{idx + 1} of {len(filenames)}")
    transcribe_one(filename, responses)

# transcribe_one("/tmp/storage/clips/front-1722009392.383259-1jcmy8-clean.png", responses)

print(responses)

json.dump(responses, open(f"transcribed/analyze_{today}.json", "w"))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/arcyleung/Projects/llm-nvr/venv/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


{'/tmp/storage/clips/front-1722009392.383259-1jcmy8-clean.png': 'In the scene, there is a person operating a green lawn mower. The person is wearing a light-colored shirt and dark pants. They are standing on the grass next to the mower, which is positioned on the sidewalk. The mower is facing towards the right side of the image.\n\nProminent vehicles in the scene include several cars parked along the street. There is a black SUV parked in front of a house, and a white van is parked on the street. There is also a white truck parked in front of a house.\n\nThe person operating the mower appears to be in the process of mowing the grass, as they are standing on the grass and the mower is turned on. The mower'}
